In [15]:
import json
from pathlib import Path
from sklearn.svm import SVR
from sentence_transformers import SentenceTransformer
import numpy as np
import joblib
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import f1_score, accuracy_score
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import Dropout


begin
Loading training data


In [ ]:
path_load_embeddings_training = "embeddings_training_Small2bis"



def flatten(list_of_list):
    return [item for sublist in list_of_list for item in sublist]

path_to_training = Path("training")
path_to_test = Path("training")  #be careful it's changed to training here

#####
# training and test sets of transcription ids
#####

# #small 1 (only to test code)
# training_set = ['ES2002']
# training_set = flatten([[m_id+s_id for s_id in 'ab'] for m_id in training_set])

# #small 2
# training_set = ['ES2002', 'ES2005', 'ES2006', 'IS1000', 'IS1001', 'IS1002', 'TS3005', 'TS3008']
# training_set = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in training_set])
# training_set.remove('IS1002a')

#small 2bis
training_set = ['ES2002', 'ES2005', 'ES2006', 'ES2007', 'ES2008', 'ES2009', 'ES2010', 'ES2015', 'ES2016', 'IS1000', 'IS1001', 'IS1002', 'IS1003' ,'IS1006', 'IS1007', 'TS3005', 'TS3008', 'TS3009', 'TS3010']
training_set = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in training_set])
training_set.remove('IS1002a')

# #small 3
# training_set = ['ES2002', 'ES2005', 'ES2006', 'ES2007', 'ES2008', 'ES2009', 'ES2010', 'ES2012', 'ES2015', 'ES2016', 'IS1000', 'IS1001', 'IS1002', 'IS1003', 'IS1005', 'IS1006', 'IS1007', 'TS3005', 'TS3008', 'TS3009', 'TS3010', 'TS3011']
# training_set = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in training_set])
# training_set.remove('IS1002a')
# training_set.remove('IS1005d')

# ## All
# training_set = ['ES2002', 'ES2005', 'ES2006', 'ES2007', 'ES2008', 'ES2009', 'ES2010', 'ES2012', 'ES2013', 'ES2015', 'ES2016', 'IS1000', 'IS1001', 'IS1002', 'IS1003', 'IS1004', 'IS1005', 'IS1006', 'IS1007', 'TS3005', 'TS3008', 'TS3009', 'TS3010', 'TS3011', 'TS3012']
# training_set = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in training_set])
# training_set.remove('IS1002a')
# training_set.remove('IS1005d')
# training_set.remove('TS3012c')






########################################################### Training #######################################################################

print("Loading training data")


#### Import labels
y_train = []
with open("training_labels.json", "r") as file:
    training_labels = json.load(file)

for transcription_id in training_set:
    y_train += training_labels[transcription_id]



#### Import embeddings
X_train = json.load(open(path_load_embeddings_training + '/X_training.json', 'r', encoding='utf-8'))
X_train_edges = json.load(open(path_load_embeddings_training + '/X_training_edges.json', 'r', encoding='utf-8'))
X_train_speaker = json.load(open(path_load_embeddings_training + '/X_training_speaker.json', 'r', encoding='utf-8'))
X_train_previous = json.load(open(path_load_embeddings_training + '/X_training_previous.json', 'r', encoding='utf-8'))


In [34]:
#### Load sentences lengths
X_train_length = []

for transcription_id in training_set:
    file_path = "training" + "/" + transcription_id + '.txt'
    with open(path_to_training / f"{transcription_id}.json", "r") as file:
        transcription = json.load(file)

    for c,utterance in enumerate(transcription):
        X_train_length.append(len(utterance["text"]))

In [10]:
path_load_embeddings_test = "embeddings_test_Small2bis_cleaned"


########################################################## Validation Data #######################################################################
print("Loading test data")


path_to_training = Path("training")
path_to_test = Path("training")  #be careful it's changed to training here

#####
# training and test sets of transcription ids
#####

# #small 1 (only to test code)
# test_set = ['ES2013']
# test_set = flatten([[m_id+s_id for s_id in 'ab'] for m_id in test_set])

# #small 2
# test_set = ['ES2013', 'IS1004', 'TS3012']
# test_set = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in test_set])
# test_set.remove('TS3012c')

#small 2 bis
test_set = ['ES2012', 'ES2013', 'IS1004','IS1005', 'TS3011', 'TS3012']
test_set = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in test_set])
test_set.remove('TS3012c')
test_set.remove('IS1005d')


y_test = []
with open("training_labels.json", "r") as file:
    training_labels = json.load(file)


for transcription_id in test_set:
    y_test += training_labels[transcription_id]




X_val = json.load(open(path_load_embeddings_test + '/X_test.json', 'r', encoding='utf-8'))
X_val_edges = json.load(open(path_load_embeddings_test + '/X_test_edges.json', 'r', encoding='utf-8'))
X_val_speaker = json.load(open(path_load_embeddings_test + '/X_test_speaker.json', 'r', encoding='utf-8'))
X_val_previous = json.load(open(path_load_embeddings_test + '/X_test_previous.json', 'r', encoding='utf-8'))


Loading test data
1
2
3
4


In [35]:
X_test_length = []
for transcription_id in test_set:
    file_path = "training" + "/" + transcription_id + '.txt'
   
    with open(path_to_test / f"{transcription_id}.json", "r") as file:
        transcription = json.load(file)
    for c,utterance in enumerate(transcription):
        X_test_length.append(len(utterance["text"]))


In [50]:
########################################################## Model #######################################################################

X_train_text = np.array(X_train_text)
X_train_title = np.array(X_train_title)
X_train_link = np.array(X_train_link)
X_train_prev_sentence = np.array(X_train_prev_sentence)
X_train_length = np.array(X_train_length)
y_train = np.array(y_train)




embedding_dim = X_train_text.shape[1]

#Input
input_text_embedding = layers.Input(shape=(embedding_dim,), name='text_embedding')
input_title_embedding = layers.Input(shape=(embedding_dim,), name='title_embedding')
input_link_embedding = layers.Input(shape=(embedding_dim,), name='link_embedding')
input_prev_sentence_embedding = layers.Input(shape=(embedding_dim,), name='prev_sentence_embedding')
input_length_embedding = layers.Input(shape=(1,), name='length_embedding')


concatenated = layers.concatenate([input_text_embedding, input_title_embedding, input_link_embedding, input_prev_sentence_embedding])



dense1 = layers.Dense(1536, activation='relu')(concatenated)
dropout1 = Dropout(0.5)(dense1)  # Ajout d'une couche de dropout avec un taux de dropout de 0.5
dense2 = layers.Dense(768, activation='relu')(dropout1)
dropout2 = Dropout(0.3)(dense2)  # Ajout d'une couche de dropout avec un taux de dropout de 0.3
dense3 = layers.Dense(256, activation='relu')(dropout2)
dropout3= Dropout(0.1)(dense3)
dense3 = layers.Dense(128, activation='relu')(dropout3)

#Output
output = layers.Dense(1, activation='sigmoid', name='output')(dense3)

#creation model
model = models.Model(inputs=[input_text_embedding, input_title_embedding, input_link_embedding, input_prev_sentence_embedding], outputs=output)

# 
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()



########################################################## Fit #######################################################################


# Convert data to numpy arrays
X_val_text = np.array(X_val_text)
X_val_title = np.array(X_val_title)
X_val_link = np.array(X_val_link)
X_val_prev_sentence = np.array(X_val_prev_sentence)
X_val_length = np.array(X_val_length)
y_val = np.array(y_val)


[-5.78094125e-02 -8.58281851e-02 -3.57198082e-02 -1.11847864e-02
  6.23635277e-02 -2.35448033e-02  6.14874549e-02  9.21988413e-02
 -5.86690381e-02 -8.79506469e-02 -6.92823008e-02  1.51864346e-02
 -5.65140182e-03  1.01023056e-02  2.20978744e-02  2.67033000e-02
 -5.22606522e-02 -1.05041072e-01 -9.69472229e-02  8.59941766e-02
 -2.42539961e-02  1.32101756e-02  3.11863068e-02  1.27339488e-04
 -6.53833225e-02 -5.93664385e-02  1.89174823e-02  6.69247955e-02
  4.58756797e-02 -4.18409631e-02 -9.08133388e-02  7.66154751e-03
  6.37946576e-02  5.10818996e-02 -5.73598966e-03 -3.75925303e-02
 -7.53389224e-02  2.21652128e-02  3.79062109e-02  7.01783895e-02
  1.06106494e-02 -4.45010476e-02 -5.41167110e-02  3.07000186e-02
  5.25758006e-02  9.30959452e-03  8.21107905e-03 -3.29816528e-02
  2.26041023e-02  3.01169585e-02 -2.00349800e-02 -6.79900423e-02
 -7.87091404e-02  1.30502470e-02  3.82726043e-02  1.20773641e-02
 -1.72328316e-02 -3.33552174e-02  1.18009955e-01 -4.37307842e-02
 -5.44748968e-03 -9.38488

In [49]:
class F1ScoreCallback(Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        # Obtenez les prédictions sur l'ensemble de validation
        val_predictions = self.model.predict(self.validation_data[0])
        val_predictions_classes = (val_predictions > 0.35).astype(int)

        # Calcul du F1 score
        val_f1 = f1_score(self.validation_data[1], val_predictions_classes)

        # Affichage du F1 score
        print(f'Epoch {epoch + 1} - Validation F1 Score: {val_f1:.4f}')





X_train_length2 = np.array([ [X_train_length[i]] for i in range(len(X_train_length))] )
X_val_length2 = np.array([[X_val_length[i]] for i in range(len(X_val_length))])



print(X_train_text.shape)
print(X_train_length2.shape)

nb_epochs = 10
# Entraînez le modèle avec le callback ModelCheckpoint
history = model.fit(
    [X_train_text, X_train_title, X_train_link, X_train_prev_sentence],
    y_train,
    epochs=nb_epochs,
    batch_size=32,
    validation_data=([X_val_text, X_val_title, X_val_link, X_val_prev_sentence], y_val),
    callbacks=[F1ScoreCallback(validation_data=([X_val_text, X_val_title, X_val_link, X_val_prev_sentence], y_val))]  
)



model.save_weights('NN_' + str(nb_epochs) + '.h5')





(57212, 384)
(57212, 1)
Epoch 1/10


ValueError: in user code:

    File "c:\Users\matze\anaconda3\envs\env_kaggle\Lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\matze\anaconda3\envs\env_kaggle\Lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\matze\anaconda3\envs\env_kaggle\Lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\matze\anaconda3\envs\env_kaggle\Lib\site-packages\keras\src\engine\training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\matze\anaconda3\envs\env_kaggle\Lib\site-packages\keras\src\engine\training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "c:\Users\matze\anaconda3\envs\env_kaggle\Lib\site-packages\keras\

In [46]:
predictions = model.predict([X_val_text, X_val_title, X_val_link, X_val_prev_sentence])


for i in range(10,65):
    binary_predictions = (predictions > i/100).astype(int)
    f1 = f1_score(binary_predictions, y_test)
    print("Pour i=",i/100, " le score F1 est: ", f1)

482/482 [==============================] - 1s 3ms/step
Pour i= 0.1  le score F1 est:  0.49697570649479417
Pour i= 0.11  le score F1 est:  0.5022747952684258
Pour i= 0.12  le score F1 est:  0.506836640279634
Pour i= 0.13  le score F1 est:  0.5113897596656217
Pour i= 0.14  le score F1 est:  0.5159235668789809
Pour i= 0.15  le score F1 est:  0.5197240784651864
Pour i= 0.16  le score F1 est:  0.5224712957900491
Pour i= 0.17  le score F1 est:  0.5266548200799646
Pour i= 0.18  le score F1 est:  0.5301558617573978
Pour i= 0.19  le score F1 est:  0.5328123204229399
Pour i= 0.2  le score F1 est:  0.5357601213394003
Pour i= 0.21  le score F1 est:  0.5362267283291831
Pour i= 0.22  le score F1 est:  0.5376836022152661
Pour i= 0.23  le score F1 est:  0.54048756584589
Pour i= 0.24  le score F1 est:  0.5412467338559164
Pour i= 0.25  le score F1 est:  0.5437706165947729
Pour i= 0.26  le score F1 est:  0.5451254202223945
Pour i= 0.27  le score F1 est:  0.5457409108806931
Pour i= 0.28  le score F1 est: 

In [78]:
################################################ Kaggle data #########################################################


import json
from pathlib import Path
from sklearn.svm import SVR
from sentence_transformers import SentenceTransformer
import numpy as np
import joblib
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import f1_score, accuracy_score

def flatten(list_of_list):
    return [item for sublist in list_of_list for item in sublist]

path_to_test = Path("test") 

# Validation Data
path_to_test = Path("test")

# Kaggle test data
test_set = ['ES2003', 'ES2004', 'ES2011', 'ES2014', 'IS1008', 'IS1009', 'TS3003', 'TS3004', 'TS3006', 'TS3007']
test_set = flatten([[m_id + s_id for s_id in 'abcd'] for m_id in test_set])

# Utterances are embedded with SentenceTransformer
bert = SentenceTransformer('all-MiniLM-L6-v2')

test_labels = {}
test_labels2 = {}
test_labels3 = {}

for transcription_id in test_set:
    X_test = []
    X_test_edges = []
    X_test_speaker = []
    X_test_previous = []
    X_test_edges.append("Beginning")

    file_path = "test" / f"{transcription_id}.txt"

    X_test_previous_number = [0]

    with open(file_path, 'r') as file:
        edge_type_transcriptionId = [words[1] for line in file if (words := line.split())]

        for words in file:
            if words:
                edge_type_transcriptionId.append(words[1])

            X_test_previous_number.append(int(words[0]))

    with open(path_to_test / f"{transcription_id}.json", "r") as file:
        transcription = json.load(file)

    X_test_text_current = []

    for c, utterance in enumerate(transcription):
        speaker_mapping = {"PM": "Project Manager", "ME": "Marketing Expert", "UI": "Interface Designer"}
        speaker = speaker_mapping.get(utterance["speaker"], "Industrial Designer")
        X_test_speaker.append(speaker)

        if c == 0:
            X_test.append(utterance["text"])
            X_test_text_current.append(utterance["text"])
        else:
            X_test.append(utterance["text"])
            X_test_edges.append(edge_type_transcriptionId[c - 1])
            X_test_text_current.append(utterance["text"])
        c += 1

    X_test_previous.append("")
    for j in range(1, len(X_test_text_current)):
        X_test_previous.append(X_test_text_current[X_test_previous_number[j]])

    X_test = bert.encode(X_test, show_progress_bar=True)
    X_test_edges = bert.encode(X_test_edges, show_progress_bar=True)
    X_test_speaker = bert.encode(X_test_speaker, show_progress_bar=True)
    X_test_previous = bert.encode(X_test_previous, show_progress_bar=True)

    X_val_text = X_test
    X_val_title = X_test_edges
    X_val_link = X_test_speaker
    X_val_prev_sentence = X_test_previous

    X_val_text = np.array(X_val_text)
    X_val_title = np.array(X_val_title)
    X_val_link = np.array(X_val_link)
    X_val_prev_sentence = np.array(X_val_prev_sentence)

    threshold = 0.32
    y_test = model.predict([X_val_text, X_val_title, X_val_link, X_val_prev_sentence])
    binary_labels = [1 if pred > threshold else 0 for pred in y_test]

    test_labels[transcription_id] = binary_labels
    print(transcription_id)
    print(binary_labels)

    threshold = 0.34
    binary_labels2 = [1 if pred > threshold else 0 for pred in y_test]

    threshold = 0.36
    binary_labels3 = [1 if pred > threshold else 0 for pred in y_test]

    test_labels[transcription_id] = binary_labels
    test_labels2[transcription_id] = binary_labels2
    test_labels3[transcription_id] = binary_labels3

with open("NN_All_larger_dropout_t32_v1.json", "w") as file:
    json.dump(test_labels, file, indent=4)

with open("NN_All_larger_dropout_t34_v1.json", "w") as file:
    json.dump(test_labels3, file, indent=4)

with open("NN_All_larger_dropout_t36_v1.json", "w") as file:
    json.dump(test_labels3, file, indent=4)


Batches: 100%|██████████| 10/10 [00:01<00:00,  9.33it/s]

10/10 [==============================] - 0s 4ms/step


ES2003a
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]


Batches: 100%|██████████| 24/24 [00:02<00:00,  8.64it/s]

 1/24 [>.............................] - ETA: 0s

...

 1/30 [>.............................] - ETA: 0s

30/30 [==============================] - 0s 5ms/step
TS3007c
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,

Batches: 100%|██████████| 46/46 [00:05<00:00,  7.88it/s]

 1/46 [..............................] - ETA: 1s

46/46 [==============================] - 0s 6ms/step
TS3007d
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,